# Imports

In [84]:
import pandas as pd
import os
import glob
import gzip

# Constants

In [74]:
frequency = 10 # samples per sec
cut_time = 5 # seconds
cut_amount = frequency * cut_time

In [75]:
features = [
    'attitude.roll', 'attitude.pitch', 'attitude.yaw',
    'gravity.x', 'gravity.y', 'gravity.z',
    'rotationRate.x', 'rotationRate.y', 'rotationRate.z',
    'userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z'
]

labels_dict = {'wlk': 0, 'sit': 1, "std": 2, "ups": 3, "jog": 4, "dws": 5}
labels = list(labels_dict.keys())

cwd = os.getcwd()
data_dir = os.path.join(cwd, 'real-data')

In [76]:
partc = 100

# Helpers

In [77]:
def cut_edges(df, amount):
    return df[amount:-amount]

In [78]:
def drop_columns(df, features):
    columns = list(df.columns)
    cols_to_drop = [col for col in columns if col not in features]
    return df.drop(cols_to_drop, axis=1)

In [79]:
def clean_df(df, amount, features):
    new_df = cut_edges(df, amount)
    new_df = drop_columns(new_df, features)
    return new_df

In [80]:
def label_file_names(label, filedir):
    return glob.glob(filedir + '/' + label + '*.csv')

# Process Data

In [81]:
frames = []
for label in labels:
    files = label_file_names(label, data_dir)
    for file in files:
        df = pd.read_csv(file)
        df = clean_df(df, cut_amount, features)
        df['partc'] = partc
        df['action'] = label
        df['action_file_index'] = labels_dict.get(label)
        frames.append(df)
full_data = pd.concat(frames)

In [86]:
with gzip.open(cwd + '/full_real_data.gz', 'wb') as f:
    f.write(full_data)

TypeError: memoryview: a bytes-like object is required, not 'DataFrame'